In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch.nn.functional as F

In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv('/kaggle/input/visual-taxonomy/train.csv')
df_test = pd.read_csv('/kaggle/input/visual-taxonomy/test.csv')

In [4]:
df_train['Category'].unique()

array(['Men Tshirts', 'Sarees', 'Kurtis', 'Women Tshirts',
       'Women Tops & Tunics'], dtype=object)

In [5]:
# df_train = df_train[:1000]

In [6]:
df_c1 = df_train[df_train['Category'] == 'Men Tshirts']
df_c2 = df_train[df_train['Category'] == 'Sarees']
df_c3 = df_train[df_train['Category'] == 'Kurtis']
df_c4 = df_train[df_train['Category'] == 'Women Tshirts']
df_c5 = df_train[df_train['Category'] == 'Women Tops & Tunics']

In [7]:
# df_c1 = df_c1[:1000]
# df_c2 = df_c2[:1000]
# df_c3 = df_c3[:1000]
# df_c4 = df_c4[:1000]
# df_c5 = df_c5[:1000]

In [8]:
# df_c1.drop(columns=['Category'])
# df_c2.drop(columns=['Category'])
# df_c3.drop(columns=['Category'])
# df_c4.drop(columns=['Category'])
# df_c5.drop(columns=['Category'])

In [9]:
df_c1.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN


<h1>Common Model Architecture</h1>

In [10]:
class LabelEncoderDict:
    def __init__(self):
        self.encoders = {}
        
    def fit(self, df, columns):
        """Fit label encoders for each column"""
        for col in columns:
            le = LabelEncoder()
            # Include NaN as a unique label by appending it to valid labels
            valid_labels = df[col].dropna().unique().tolist()
            valid_labels.append('NaN')  # Assign a label for NaN
            le.fit(valid_labels)
            self.encoders[col] = le
            
    def transform(self, df, columns):
        """Transform labels using fitted encoders"""
        encoded = np.zeros((len(df), len(columns)))
        for i, col in enumerate(columns):
            series = df[col].copy()
            # Replace NaNs with the string 'NaN' so they can be encoded
            series = series.fillna('NaN')
            encoded[:, i] = self.encoders[col].transform(series)
        return encoded
    
    def get_num_classes(self, column):
        """Get number of classes for a specific column"""
        return len(self.encoders[column].classes_)


class MultiLabelImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, attr_columns=10):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
        # TODO: Changes 
        self.attr_columns = attr_columns
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Get image path
        img_name = str(self.df.iloc[idx]['id']).zfill(6)
        img_path = os.path.join(self.image_dir, f"{img_name}.jpg")
        
        # Load image
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (224, 224))  # TODO: Changes with (512, 512)
        
        if self.transform:
            image = self.transform(image)
        
        # Ensure labels are integers and convert to tensor
        labels = torch.tensor(self.df.iloc[idx][self.attr_columns].astype(int).values, dtype=torch.long)
        
        return image, labels


def prepare_data(df, image_dir, batch_size=32, test_size=0.2, num_attr_columns=10):
    """
    Prepare data loaders and label encoders
    """
    # Define attribute columns
    attr_columns = [f'attr_{i}' for i in range(1, num_attr_columns+1)] # TODO: Changes with the number of column to consider
    
    # Create and fit label encoders
    label_encoders = LabelEncoderDict()
    label_encoders.fit(df, attr_columns)
    
    # Transform labels
    encoded_labels = label_encoders.transform(df, attr_columns)
    df_encoded = df.copy()
    for i, col in enumerate(attr_columns):
        df_encoded[col] = encoded_labels[:, i]
    
    # Split data
    train_df, val_df = train_test_split(df_encoded, test_size=test_size, random_state=42)
    
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((256, 512)),  # TODO: Changes with (512, 512)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create datasets
    train_dataset = MultiLabelImageDataset(
        train_df,
        image_dir,
        transform=transform,
        attr_columns=attr_columns
    )
    
    val_dataset = MultiLabelImageDataset(
        val_df,
        image_dir,
        transform=transform,
        attr_columns=attr_columns
    )
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Get number of classes for each attribute
    num_classes_per_attr = [label_encoders.get_num_classes(col) for col in attr_columns]
    
    return train_loader, val_loader, label_encoders, num_classes_per_attr

class MultiLabelClassifier(nn.Module):
    def __init__(self, num_classes_per_attr, pretrained=True):
        super(MultiLabelClassifier, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained) #Backbone model 1
        num_features = self.backbone.fc.in_features
        self.backbone = torch.nn.Sequential(*(list(self.backbone.children())[:-1]))
        
        # Create separate classifier heads for each attribute
        self.classifier_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(num_features, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, num_classes)
            ) for num_classes in num_classes_per_attr
        ])

    def forward(self, x):
        features = self.backbone(x)
        features = features.view(features.size(0), -1)
        return [head(features) for head in self.classifier_heads]

class MultiLabelCELoss(nn.Module):
    def __init__(self):
        super(MultiLabelCELoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        # outputs is a list of predictions for each label
        # targets is a tensor of shape (batch_size, num_labels)
        loss = 0
        for i, output in enumerate(outputs):
            loss += self.criterion(output, targets[:, i])
        return loss / len(outputs)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class MultiLabelClassifier(nn.Module):
    def __init__(self, num_classes_per_attr, pretrained=True):
        super(MultiLabelClassifier, self).__init__()
        self.backbone = models.efficientnet_b2(pretrained=pretrained)
        num_features = self.backbone.classifier[1].in_features
        self.backbone = torch.nn.Sequential(*(list(self.backbone.children())[:-1]))
        
        self.feature_extractor = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.BatchNorm1d(num_features),
            nn.Dropout(0.5),
            nn.Linear(num_features, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.4)
        )
        
        self.shared_features = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3)
        )
        
        self.classifier_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.BatchNorm1d(256),
                nn.Dropout(0.2),
                nn.Linear(256, num_classes)
            ) for num_classes in num_classes_per_attr
        ])

    def forward(self, x):
        features = self.backbone(x)
        features = self.feature_extractor(features)
        shared = self.shared_features(features)
        
        return [head(shared) for head in self.classifier_heads]

class MultiLabelCELoss(nn.Module):
    def __init__(self):
        super(MultiLabelCELoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        loss = 0
        for i, output in enumerate(outputs):
            loss += self.criterion(output, targets[:, i])
        return loss / len(outputs)

def train_model(model, train_loader, val_loader, num_epochs, num_classes_per_attr, model_type):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = MultiLabelCELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=1e-3,
        epochs=num_epochs,
        steps_per_epoch=len(train_loader)
    )
    
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_predictions = [0] * len(num_classes_per_attr)
        total_predictions = 0
        overall_correct = 0  # To track instances where all labels match
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
            
            # Check predictions for each attribute
            all_labels_match = torch.ones(labels.size(0), dtype=torch.bool, device=device)  # For overall accuracy
            for i, output in enumerate(outputs):
                _, predicted = torch.max(output, 1)
                correct_predictions[i] += (predicted == labels[:, i]).sum().item()
                all_labels_match &= (predicted == labels[:, i])  # Track overall label match for this instance
                
            overall_correct += all_labels_match.sum().item()
            total_predictions += labels.size(0)
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct_predictions = [0] * len(num_classes_per_attr)
        val_total_predictions = 0
        val_overall_correct = 0  # For validation overall accuracy
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                all_labels_match_val = torch.ones(labels.size(0), dtype=torch.bool, device=device)  # For overall accuracy
                for i, output in enumerate(outputs):
                    _, predicted = torch.max(output, 1)
                    val_correct_predictions[i] += (predicted == labels[:, i]).sum().item()
                    all_labels_match_val &= (predicted == labels[:, i])  # Track overall label match
                
                val_overall_correct += all_labels_match_val.sum().item()
                val_total_predictions += labels.size(0)
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        
        # Print accuracy for each attribute
        for i in range(len(num_classes_per_attr)):
            train_acc = 100 * correct_predictions[i] / total_predictions
            val_acc = 100 * val_correct_predictions[i] / val_total_predictions
            print(f'Attribute {i+1} - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')
        
        # Overall accuracy where all predicted labels match target labels
        overall_train_acc = 100 * overall_correct / total_predictions
        overall_val_acc = 100 * val_overall_correct / val_total_predictions
        print(f'Overall Train Accuracy: {overall_train_acc:.2f}%')
        print(f'Overall Validation Accuracy: {overall_val_acc:.2f}%')
        
        # Check for improvement and early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f'best_model_{model_type}.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break


In [14]:
import pickle
import gc

def main(df_c1, df_c2, df_c3, df_c4, df_c5):
    # Set image directory
    image_dir = '/kaggle/input/visual-taxonomy/train_images'
    
    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Define a function to train each model sequentially
    def train_single_model(data, num_attr_columns, model_type):
        # Prepare data
        print(f"Preparing data for {model_type}")
        train_loader, val_loader, label_encoders, num_classes_per_attr = prepare_data(data, image_dir, batch_size=32, num_attr_columns=num_attr_columns)
        
        # Initialize the model
        print(f"Initializing model {model_type}")
        model = MultiLabelClassifier(num_classes_per_attr).to(device)
        
        # Define loss and optimizer
        criterion = MultiLabelCELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Train the model
        print(f"Training model {model_type}")
        train_model(model, train_loader, val_loader, num_epochs=5, num_classes_per_attr=num_classes_per_attr, model_type=model_type)
        
        # Save label encoders
        with open(f'label_encoders_{model_type}.pkl', 'wb') as f:
            pickle.dump(label_encoders, f)

        print("---------------------------------------------")
        print(f"number of classes for {model_type} is {num_classes_per_attr}")
        print("---------------------------------------------")

        # Free up memory
        del model, train_loader, val_loader, label_encoders, num_classes_per_attr
        torch.cuda.empty_cache()
        gc.collect()
    
    # Train models one at a time
    train_single_model(df_c1, num_attr_columns=5, model_type="c1")
    train_single_model(df_c2, num_attr_columns=10, model_type="c2")
    train_single_model(df_c3, num_attr_columns=9, model_type="c3")
    train_single_model(df_c4, num_attr_columns=8, model_type="c4")
    train_single_model(df_c5, num_attr_columns=10, model_type="c5")

In [15]:
main(df_c1, df_c2, df_c3, df_c4, df_c5)

Preparing data for c1
Initializing model c1
Training model c1
Epoch 1/1
Training Loss: 0.9935
Validation Loss: 0.7883
Attribute 1 - Train Acc: 39.38%, Val Acc: 52.00%
Attribute 2 - Train Acc: 71.50%, Val Acc: 90.00%
Attribute 3 - Train Acc: 67.88%, Val Acc: 90.50%
Attribute 4 - Train Acc: 64.38%, Val Acc: 88.50%
Attribute 5 - Train Acc: 40.12%, Val Acc: 14.50%
Overall Train Accuracy: 11.38%
Overall Validation Accuracy: 9.00%
---------------------------------------------
number of classes for c1 is [5, 3, 3, 4, 3]
---------------------------------------------
Preparing data for c2
Initializing model c2
Training model c2
Epoch 1/1
Training Loss: 1.6006
Validation Loss: 1.3996
Attribute 1 - Train Acc: 37.75%, Val Acc: 61.00%
Attribute 2 - Train Acc: 35.00%, Val Acc: 61.00%
Attribute 3 - Train Acc: 56.12%, Val Acc: 75.50%
Attribute 4 - Train Acc: 30.25%, Val Acc: 49.50%
Attribute 5 - Train Acc: 35.00%, Val Acc: 65.00%
Attribute 6 - Train Acc: 34.50%, Val Acc: 58.00%
Attribute 7 - Train Acc

In [16]:
def get_num_classes_per_attr(df, num_attr_columns, df_name=None):
    """
    Takes a DataFrame and the number of attribute columns to process.
    
    Args:
    - df: The DataFrame.
    - num_attr_columns: The number of attribute columns to consider.
    - df_name: Optional name of the DataFrame, used to apply specific fixes.
    
    Returns:
    - A list containing the number of unique classes for each attribute, adding 1 if NaN values are present.
    """
    # Select the first 'num_attr_columns' columns from the DataFrame
    attr_columns = [f'attr_{i}' for i in range(1, num_attr_columns + 1)]
    
    # Count the unique values in each attribute column, adding 1 if NaN values are present
    num_classes_per_attr = [
        df[attr].nunique() + (1 if df[attr].isnull().any() else 0) for attr in attr_columns
    ]
    
    # Specific fix for df_c3 to correct counting for certain columns
    if df_name == "df_c3":
        # Force the number of unique classes for attr_5, attr_6, attr_7 to be 3 (based on prior knowledge)
        num_classes_per_attr[4] = 3
        num_classes_per_attr[5] = 3
        num_classes_per_attr[6] = 3
    
    return num_classes_per_attr

def get_all_num_classes_per_attr(df_list, attr_num_list, df_names):
    """
    Takes a list of DataFrames and a corresponding list of the number of attribute columns for each.
    
    Args:
    - df_list: List of DataFrames.
    - attr_num_list: List of integers representing the number of attributes for each DataFrame.
    - df_names: List of DataFrame names, used for specific fixes.
    
    Returns:
    - A list containing the number of unique classes for each attribute for each DataFrame.
    """
    all_num_classes = []
    
    # Iterate over each DataFrame and corresponding number of attribute columns
    for df, num_attr_columns, df_name in zip(df_list, attr_num_list, df_names):
        num_classes = get_num_classes_per_attr(df, num_attr_columns, df_name)
        all_num_classes.append(num_classes)
    
    return all_num_classes

# Example usage:
# Assuming df_c1 to df_c5 are your DataFrames
df_list = [df_c1, df_c2, df_c3, df_c4, df_c5]
attr_num_list = [5, 10, 9, 8, 10]  # Number of attribute columns to consider for each DataFrame
df_names = ["df_c1", "df_c2", "df_c3", "df_c4", "df_c5"]

num_classes_per_attr_all = get_all_num_classes_per_attr(df_list, attr_num_list, df_names)

# This will give you a list of lists, where each sublist contains the num_classes_per_attr for each DataFrame.
for i, num_classes in enumerate(num_classes_per_attr_all):
    print(f"num_classes_per_attr for df_c{i + 1}: {num_classes}")


num_classes_per_attr for df_c1: [5, 3, 3, 4, 3]
num_classes_per_attr for df_c2: [5, 7, 4, 9, 5, 4, 5, 6, 10, 3]
num_classes_per_attr for df_c3: [14, 3, 3, 3, 3, 3, 3, 4, 3]
num_classes_per_attr for df_c4: [8, 4, 4, 4, 7, 4, 3, 1]
num_classes_per_attr for df_c5: [13, 5, 3, 8, 3, 4, 7, 5, 5, 7]


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import pickle

def load_model(model_path, num_classes_per_attr, device):
    # Initialize the model architecture and load the saved weights
    model = MultiLabelClassifier(num_classes_per_attr)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def load_label_encoders(encoder_path):
    with open(encoder_path, 'rb') as f:
        encoders = pickle.load(f)
    return encoders


def preprocess_image(image_path, image_size=(224, 224)):
    # Define image transformations (same as used during training)
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize based on ImageNet
    ])
    
    # Open image and apply transformations
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def inference(image_path, model_path, encoder_path, num_classes_per_attr, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Load the model and encoders
    model = load_model(model_path, num_classes_per_attr, device)
    label_encoders = load_label_encoders(encoder_path)

    # Preprocess the image
    image = preprocess_image(image_path).to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(image)

    # Decode predictions
    predicted_labels = []
    for i, output in enumerate(outputs):
        _, predicted = torch.max(output, 1)

        # Attribute name should match the encoder dictionary keys like 'attr_1', 'attr_2', etc.
        attr_name = f'attr_{i+1}'
        if attr_name in label_encoders.encoders:
            decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
            predicted_labels.append(decoded_label)
        else:
            raise KeyError(f"Encoder for {attr_name} not found in the loaded label encoders.")
    
    return predicted_labels


# Example usage:
image_path = "/kaggle/input/visual-taxonomy/train_images/000001.jpg"
model_path = "best_model_c1.pth"
encoder_path = "/kaggle/working/label_encoders_c1.pkl"
num_classes_per_attr = [5, 3, 3, 4, 3]
predictions = inference(image_path, model_path, encoder_path, num_classes_per_attr)
print("Predicted labels:", predictions)


In [17]:
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
import pickle
import csv

# Map categories to corresponding attribute columns (e.g., c1, c2, c3)
category_to_attributes = {
    'Men Tshirts': 'c1',
    'Sarees': 'c2',
    'Kurtis': 'c3',
    'Women Tshirts': 'c4',
    'Women Tops & Tunics': 'c5',
}

# Dummy value for missing attributes
DUMMY_VALUE = "dummy_value"

def load_model(model_path, num_classes_per_attr, device):
    model = MultiLabelClassifier(num_classes_per_attr)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def load_label_encoders(encoder_path):
    with open(encoder_path, 'rb') as f:
        encoders = pickle.load(f)
    return encoders

def preprocess_image(image_path, image_size=(224, 224)):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def inference(image_path, model, label_encoders, num_classes_per_attr, device='cuda' if torch.cuda.is_available() else 'cpu'):
    image = preprocess_image(image_path).to(device)
    with torch.no_grad():
        outputs = model(image)

    predicted_labels = []
    for i, output in enumerate(outputs):
        _, predicted = torch.max(output, 1)
        attr_name = f'attr_{i+1}'
        if attr_name in label_encoders.encoders:
            decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
            predicted_labels.append(decoded_label)
        else:
            predicted_labels.append(DUMMY_VALUE)
    
    return predicted_labels

def make_predictions_for_dataset(df, model_paths, encoder_paths, output_path, image_dir, num_classes_per_attr_list, device='cuda' if torch.cuda.is_available() else 'cpu'):
    
    all_models = []
    all_label_encoders = []
    
    # Load models and encoders for each attribute (c1 to c5)
    for model_path, encoder_path, num_classes_per_attr in zip(model_paths, encoder_paths, num_classes_per_attr_list):
        model = load_model(model_path, num_classes_per_attr, device)
        label_encoders = load_label_encoders(encoder_path)
        all_models.append(model)
        all_label_encoders.append(label_encoders)
    
    all_predictions = []
    
    for idx, row in df.iterrows():
        category = row['Category']
        attribute_key = category_to_attributes.get(category, None)

        if attribute_key:
            # Determine which model and encoder to use based on the category
            attribute_idx = int(attribute_key[1]) - 1  # Example: 'c1' -> index 0, 'c2' -> index 1, etc.
            model = all_models[attribute_idx]
            label_encoders = all_label_encoders[attribute_idx]
            num_classes_per_attr = num_classes_per_attr_list[attribute_idx]

            image_path = f"{image_dir}/{str(row['id']).zfill(6)}.jpg"  # Assuming image names follow a pattern with id
            
            try:
                predictions = inference(image_path, model, label_encoders, num_classes_per_attr, device)
            except Exception as e:
                print(f"Error with image {image_path}: {e}")
                predictions = [DUMMY_VALUE] * len(num_classes_per_attr)  # Fallback in case of error
        else:
            predictions = [DUMMY_VALUE] * len(num_classes_per_attr_list[0])  # Handle if category has no mapping
            
        # Pad the predictions to ensure each row has 10 attributes
        while len(predictions) < 10:
            predictions.append(DUMMY_VALUE)
        
        all_predictions.append([row['id']] + predictions)

    # Save predictions to CSV
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header = ['id'] + [f'c{i+1}' for i in range(10)]  # Fixed number of attributes as 10
        writer.writerow(header)
        writer.writerows(all_predictions)

    print(f"Predictions saved to {output_path}")

# Example usage
dataset_path = '/kaggle/input/visual-taxonomy/test.csv'  # Path to your dataset
model_paths = ['/kaggle/working/best_model_c1.pth', '/kaggle/working/best_model_c2.pth', '/kaggle/working/best_model_c3.pth', '/kaggle/working/best_model_c4.pth', '/kaggle/working/best_model_c5.pth']
encoder_paths = ['/kaggle/working/label_encoders_c1.pkl', '/kaggle/working/label_encoders_c2.pkl', '/kaggle/working/label_encoders_c3.pkl', '/kaggle/working/label_encoders_c4.pkl', '/kaggle/working/label_encoders_c5.pkl']
output_path = '/kaggle/working/output_predictions.csv'  # Path to save predictions
image_dir = '/kaggle/input/visual-taxonomy/test_images'  # Directory containing images
num_classes_per_attr_list = [
    [5, 3, 3, 4, 3],  # For c1
    [5, 7, 4, 9, 5, 4, 5, 6, 10, 3],  # For c2
    [14, 3, 3, 3, 3, 3, 3, 4, 3],  # For c3
    [8, 4, 4, 4, 7, 4, 3, 1],  # For c4
    [13, 5, 3, 8, 3, 4, 7, 5, 5, 7],  # For c5
]

test_df = pd.read_csv(dataset_path)
# test_df = test_df[:300]

make_predictions_for_dataset(test_df, model_paths, encoder_paths, output_path, image_dir, num_classes_per_attr_list)


Predictions saved to /kaggle/working/output_predictions.csv
